# In Class Notebook, Week 14

You can always paste the URL of this notebook (https://github.com/UIUC-iSchool-DataViz/is445_spring2021/blob/master/week14/inClass_week14.ipynb ) into the nbviewer interface for a plain-text rendering:

https://kokes.github.io/nbviewer.js/viewer.html

Let me write something!

Before yt, let's check out ipyvolume

## ipyvolume

In [1]:
# import our usual things
import numpy as np
import pandas as pd
import ipyvolume

In [2]:
x, y, z = np.random.random((3, 100)) # 3 dimensions, 100 points total

In [7]:
ipyvolume.quickscatter?

In [8]:
# quick 3d scatter plot
ipyvolume.quickscatter(x,y,z, size=5, marker='diamond')

Now using "real" data -- data from a VERY low resolution simulation of 2 galaxies colliding. (Galaxies are collections of billions of stars, we live in the Milky Way galaxy, our star (the Sun!) is about 1/2 out on a disk of stars in the Milky Way).

In [9]:
galaxy_file = '/Users/jillnaiman/Downloads/outarrsnap_001_fac1n3.txt'

We note: the data is super confusing to look at!  Luckily there is a data reader -- solverlibs.py -- included in the galaxyFiles.zip!

In [10]:
# to load solver -- can copy to current directory

# or update path:
from sys import path
path.append('/Users/jillnaiman/Downloads/')

In [11]:
path

['/Users/jillnaiman/is445_spring2021/week14',
 '/opt/anaconda3/envs/DataViz/lib/python37.zip',
 '/opt/anaconda3/envs/DataViz/lib/python3.7',
 '/opt/anaconda3/envs/DataViz/lib/python3.7/lib-dynload',
 '',
 '/Users/jillnaiman/.local/lib/python3.7/site-packages',
 '/opt/anaconda3/envs/DataViz/lib/python3.7/site-packages',
 '/opt/anaconda3/envs/DataViz/lib/python3.7/site-packages/IPython/extensions',
 '/Users/jillnaiman/.ipython',
 '/Users/jillnaiman/Downloads/']

In [12]:
from solverlibs import read_in_galaxy_data

In [13]:
read_in_galaxy_data?

In [14]:
# Note: a terrible person has not included doc strings!!!

In [15]:
t_h, r_h, v_h, e_h, N, part_type = read_in_galaxy_data(galaxy_file)

In [17]:
r_h.shape # 82 particles, 3 positions -- x/y/z, 500 time steps

(82, 3, 500)

In [18]:
N

82